
# Conversión de Cajas de Porcentaje a Metros y Cálculo de Estadísticas

Este cuaderno realiza los siguientes pasos:

1. Carga un archivo CSV con columnas `xmin`, `ymin`, `xmax`, `ymax` en valores porcentuales (0–100).
2. Convierte estas coordenadas a metros, utilizando las dimensiones de la imagen y la relación de conversión (1 píxel = 3.197 cm).
3. Calcula el ancho (`width_m`), alto (`height_m`) y área (`area_m2`) en metros de cada caja.
4. Guarda un nuevo CSV con las columnas originales más las columnas convertidas a metros.
5. Calcula estadísticas de las cajas:
   - Caja(s) con ancho y alto mínimos simultáneamente.
   - Caja(s) con ancho y alto máximos simultáneamente.
   - Caja(s) con área mínima.
   - Caja(s) con área máxima.


In [1]:

import pandas as pd
import os
from IPython.display import display

# Verificar existencia del archivo CSV
input_path = "labels_tifname_porcentual.csv"  # Ajustar si el nombre o ruta cambian
if not os.path.isfile(input_path):
    raise FileNotFoundError(f"No se encontró el archivo CSV en la ruta: {input_path}")


In [2]:

# Dimensiones de la imagen en píxeles
img_width_pixels = 2947   # Ancho de la imagen en píxeles
img_height_pixels = 3054  # Alto de la imagen en píxeles

# Conversión de píxel a centímetros
px_to_cm = 3.197000000000000536  # Centímetros por píxel


In [3]:

def percent_x_to_m(x_percent):
    """
    Convierte un porcentaje en el eje X (0-100) a metros.
    1. Calcula píxeles: (x_percent / 100) * ancho_imagen_píxeles
    2. Convierte píxeles a cm: píxeles * px_to_cm
    3. Convierte cm a metros: dividir por 100
    """
    x_pixels = (x_percent / 100.0) * img_width_pixels
    x_cm = x_pixels * px_to_cm
    return x_cm / 100.0  # Convertir de cm a metros

def percent_y_to_m(y_percent):
    """
    Convierte un porcentaje en el eje Y (0-100) a metros.
    1. Calcula píxeles: (y_percent / 100) * alto_imagen_píxeles
    2. Convierte píxeles a cm: píxeles * px_to_cm
    3. Convierte cm a metros: dividir por 100
    """
    y_pixels = (y_percent / 100.0) * img_height_pixels
    y_cm = y_pixels * px_to_cm
    return y_cm / 100.0  # Convertir de cm a metros


In [4]:

# Cargar el CSV original
df = pd.read_csv(input_path)

# Crear nuevas columnas con coordenadas convertidas a metros
df["xmin_m"] = df["xmin"].apply(percent_x_to_m)
df["xmax_m"] = df["xmax"].apply(percent_x_to_m)
df["ymin_m"] = df["ymin"].apply(percent_y_to_m)
df["ymax_m"] = df["ymax"].apply(percent_y_to_m)

# Calcular ancho (width), alto (height) y área en metros
df["width_m"] = df["xmax_m"] - df["xmin_m"]
df["height_m"] = df["ymax_m"] - df["ymin_m"]
df["area_m2"] = df["width_m"] * df["height_m"]

# Guardar el nuevo CSV con dimensiones en metros
output_path = "labels_absolute_res1x_converted.csv"
df.to_csv(output_path, index=False)
print(f"Archivo convertido guardado en: {output_path}")


Archivo convertido guardado en: labels_absolute_res1x_converted.csv


In [9]:
# 1. Calcular área mínima y máxima para cada categoría ("label")
resumen_areas = df.groupby("label").agg(
    area_min_m2=("area_m2", "min"),
    area_max_m2=("area_m2", "max")
).reset_index()

# 2. Mostrar la tabla resumen
print("📊 Resumen de área mínima y máxima por categoría:")
display(resumen_areas)

📊 Resumen de área mínima y máxima por categoría:


,label,area_min_m2,area_max_m2
0,Lemon,0.021643,44.891561
1,Orange,0.008120,67.843846
2,Tree,0.506173,554.359404


In [10]:
# 1. Agrupar por etiqueta y calcular las estadísticas solicitadas
resumen_estadisticas = df.groupby("label").agg(
    # Área mínima y máxima
    area_min_m2   = ("area_m2", "min"),
    area_max_m2   = ("area_m2", "max"),
    # Ancho (width) mínimo, máximo y promedio
    width_min_m   = ("width_m", "min"),
    width_max_m   = ("width_m", "max"),
    width_mean_m  = ("width_m", "mean"),
    # Alto (height) mínimo, máximo y promedio
    height_min_m  = ("height_m", "min"),
    height_max_m  = ("height_m", "max"),
    height_mean_m = ("height_m", "mean")
).reset_index()

# 2. Opcional: redondear valores numéricos a cierta cantidad de decimales
#    para facilitar la lectura (por ejemplo, 4 decimales)
resumen_estadisticas[[
    "area_min_m2", "area_max_m2",
    "width_min_m", "width_max_m", "width_mean_m",
    "height_min_m", "height_max_m", "height_mean_m"
]] = resumen_estadisticas[[
    "area_min_m2", "area_max_m2",
    "width_min_m", "width_max_m", "width_mean_m",
    "height_min_m", "height_max_m", "height_mean_m"
]].round(4)

# 3. Mostrar la tabla resumen
print("📊 Resumen de estadísticas por categoría:")
display(resumen_estadisticas)

📊 Resumen de estadísticas por categoría:


,label,area_min_m2,area_max_m2,width_min_m,width_max_m,width_mean_m,height_min_m,height_max_m,height_mean_m
0,Lemon,0.0216,44.8916,0.1699,6.1150,2.8732,0.1274,7.4330,2.9874
1,Orange,0.0081,67.8438,0.1299,8.7697,4.2404,0.0376,8.2036,4.6003
2,Tree,0.5062,554.3594,0.3516,22.7013,4.0043,0.7674,24.4197,4.1954


In [7]:
df = pd.read_csv("labels_absolute_res1x_converted.csv")
df.head(10)

,image_path,xmin,ymin,xmax,ymax,label,xmin_m,xmax_m,ymin_m,ymax_m,width_m,height_m,area_m2
0,database_batch_1.tif,0.046181,4.456323e-02,1.092956,1.782529,Lemon,0.043510,1.029735,4.350993e-02,1.740397,0.986225,1.696887,1.673512
1,database_batch_1.tif,3.094143,2.638673e-17,6.326830,1.574567,Lemon,2.915165,5.960860,2.576305e-17,1.537351,3.045695,1.537351,4.682302
2,database_batch_1.tif,8.451167,0.000000e+00,11.683856,1.247770,Lemon,7.962316,11.008014,0.000000e+00,1.218278,3.045698,1.218278,3.710506
3,database_batch_1.tif,1.831856,5.481277e+00,3.632924,7.100408,Lemon,1.725894,3.422781,5.351721e+00,6.932581,1.696887,1.580861,2.682542
4,database_batch_1.tif,5.865017,3.312534e+00,10.498535,7.709439,Lemon,5.525761,9.891256,3.234238e+00,7.527217,4.365496,4.292979,18.740984
5,database_batch_1.tif,11.791609,3.461078e+00,14.131459,5.852638,Lemon,11.109534,13.314037,3.379271e+00,5.714304,2.204503,2.335033,5.147586
6,database_batch_1.tif,17.133239,3.475932e+00,19.272970,5.303024,Lemon,16.142182,18.158142,3.393774e+00,5.177681,2.015960,1.783907,3.596285
7,database_batch_1.tif,15.270596,8.897792e+00,17.595052,11.512168,Lemon,14.387282,16.577282,8.687482e+00,11.240064,2.190000,2.552582,5.590154
8,database_batch_1.tif,20.157255,8.264005e+00,22.404744,10.447602,Lemon,18.991277,21.108761,8.068675e+00,10.200661,2.117484,2.131986,4.514446
9,database_batch_1.tif,13.300185,3.958010e-17,15.501497,0.891264,Lemon,12.530848,14.604827,3.864458e-17,0.870197,2.073979,0.870197,1.804772
